In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
all_drives = pd.read_csv('../data/espn_drives2009-2016.csv')

In [3]:
all_drives.head(5)

,Unnamed: 0,away,away_score_after,drive,home,home_score_after,offense,plays,result,time,yds_gained,gameId
0,0,PIT,0,0,TEN,0,PIT,3,Punt,1:44,2,290910023
1,1,PIT,0,1,TEN,0,TEN,3,Punt,1:52,2,290910023
2,2,PIT,0,2,TEN,0,PIT,5,Punt,3:04,2,290910023
3,3,PIT,0,3,TEN,0,TEN,6,Missed FG,1:35,38,290910023
4,4,PIT,0,4,TEN,0,PIT,3,Punt,1:55,-6,290910023


In [4]:
all_drives['result'].unique()

array(['Punt', 'Missed FG', 'Intercepted Pass', 'Touchdown', 'Fumble',
       'Field Goal', 'End of Half', 'End of Game', 'Downs',
       'End of 1st Half', 'Joshua Cribbs 67 Yard Punt Return',
       'Adrian Peterson 64 Yard Run', 'JAC Timeout', 'NO Timeout',
       'Blocked FG', 'Fumble Touchdown',
       'Desean Jackson 85 Yard Punt Return',
       'Brent Celek 9 Yard Pass From Donovan Mcnabb',
       'Blocked Punt Touchdown', 'Intercepted Pass Touchdown',
       'Julius Jones 62 Yard Run', 'Safety', 'Ryan Grant 3 Yard Run',
       'Blocked Punt', 'Adrian Peterson 27 Yard Run', 'Fumble, Safety',
       'Blocked FG Touchdown',
       'Cadillac Williams 8 Yard Pass From Byron Leftwich', 'BUF Timeout',
       'Frank Gore 79 Yard Run', 'Allen Rossum 67 Yard Punt Return',
       'Will Blackmon 0 Yard Kickoff Return, Tackled In Own End Zone',
       'Percy Harvin 101 Yard Kickoff Return',
       'Ellis Hobbs 0 Yard Kickoff Return, Tackled In Own End Zone',
       'Penalty on Anthony Madis

In [5]:
all_drives['uid'] = all_drives['gameId'].astype(str) +"-"+ all_drives['drive'].astype(str)

In [6]:
all_drives.sample(15)

,Unnamed: 0,away,away_score_after,drive,home,home_score_after,offense,plays,result,time,yds_gained,gameId,uid
41007,23,WSH,16,23,DAL,19,WSH,0,End of Game,0:09,0,400791686,400791686-23
3624,16,JAX,10,17,BUF,15,JAX,3,Punt,2:09,8,291122030,291122030-17
48195,12,DEN,17,13,OAK,0,DEN,10,Touchdown,4:46,64,400874694,400874694-13
15217,5,NE,0,5,NYG,0,NE,7,Intercepted Pass,3:57,63,311106017,311106017-5
34043,12,CHI,14,13,MIN,10,MIN,6,Punt,1:40,16,400554340,400554340-13
47985,13,TEN,14,14,HOU,0,HOU,13,Touchdown,4:29,81,400874632,400874632-14
1113,7,CHI,14,7,DET,14,CHI,4,Touchdown,1:43,7,291004003,291004003-7
36185,23,KC,19,24,SD,7,SD,9,Interception,2:23,51,400554413,400554413-24
27015,2,DET,0,2,DAL,0,DET,5,Punt,3:24,20,331027008,331027008-2
14344,21,CHI,39,22,MIN,10,CHI,5,Punt,3:09,21,311016003,311016003-22


In [7]:
# Write a function to count drives in a given dF based on outcome
# Types: TD, FG, Turnover, Punt, End of half/game, Shenanigans
# maybe make columns for each of those types except shenanigans
def find_tds(result_list):
    '''Accepts a list of drive outcomes. 
    Return a binary list of which result in touchdowns'''
    
    is_td = []
    for outcome in [x.lower() for x in result_list]:
        td = 0
        if "touchdown" in outcome:
            if "intercept" in outcome:
                td = 0
            elif "fumble" in outcome:
                td = 0
            elif "punt" in outcome:
                td = 0
            else:
                td = 1
                
        is_td.append(td)
        
    return is_td

def find_fgs(result_list):
    '''Accepts a list of drive outcomes. 
    Return a binary list of which result in field goals'''
    
    is_fg = []
    for outcome in [x.lower() for x in result_list]:
        fg = 0
        if ("field goal" in outcome) or ("fg" in outcome):
            if "block" in outcome:
                fg = 0
            else:
                fg = 1
                
        is_fg.append(fg)
        
    return is_fg

def find_punts(result_list):
    is_punt = []
    for outcome in [x.lower() for x in result_list]:
        punt = 0
        if "punt" in outcome:
            punt = 1
        is_punt.append(punt)
        
    return is_punt
    
def find_turnovers(result_list):
    is_to = []
    for outcome in [x.lower() for x in result_list]:
        to = 0
        if "intercept" in outcome:
            to = 1
        elif "fumble" in outcome:
            to = 1
        elif "downs" in outcome:
            to = 1
        elif "safety" in outcome:
            to = 1
        is_to.append(to)
        
    return is_to
    
def find_endofhalves(result_list):
    is_eoh = []
    for outcome in [x.lower() for x in result_list]:
        eoh = 0
        if "end of half" in outcome:
            eoh = 1
        elif "end of game" in outcome:
            eoh = 1
        is_eoh.append(eoh)
        
    return is_eoh

In [8]:
all_drives['TD'] = find_tds(all_drives['result'].values)

In [9]:
all_drives['FG'] = find_fgs(all_drives['result'].values)
all_drives['punt'] = find_punts(all_drives['result'].values)
all_drives['turnover'] = find_turnovers(all_drives['result'].values)
all_drives['EoH'] = find_endofhalves(all_drives['result'].values)

In [10]:
all_drives.sample(10)

,Unnamed: 0,away,away_score_after,drive,home,home_score_after,offense,plays,result,time,yds_gained,gameId,uid,TD,FG,punt,turnover,EoH
46206,4,BUF,0,4,JAX,7,JAX,5,Punt,2:38,13,400874554,400874554-4,0,0,1,0,0
1380,23,DEN,17,24,DAL,10,DEN,2,End of Game,0:01,-1,291004007,291004007-24,0,0,0,0,1
13974,14,SF,31,15,TB,3,TB,7,Downs,2:41,62,311009025,311009025-15,0,0,0,1,0
13028,9,CIN,3,9,SF,0,SF,9,Punt,4:50,33,310925004,310925004-9,0,0,1,0,0
2125,16,CLE,3,17,GB,31,CLE,3,Punt,1:13,1,291025005,291025005-17,0,0,1,0,0
39201,1,DAL,0,1,SEA,3,DAL,15,Field Goal,8:04,65,400791681,400791681-1,0,1,0,0,0
20627,0,TB,0,0,NO,0,NO,4,Intercepted Pass,2:33,48,321021027,321021027-0,0,0,0,1,0
29458,17,ATL,24,18,WSH,20,WSH,3,Punt,2:05,4,331215001,331215001-18,0,0,1,0,0
23737,26,GB,55,27,TEN,0,TEN,6,Touchdown,1:12,53,321223009,321223009-27,1,0,0,0,0
37793,7,WSH,13,7,PHI,0,PHI,3,Fumble,1:15,47,400791704,400791704-7,0,0,0,1,0


In [12]:
# Summarize a game's drives
subdf = all_drives[ all_drives['gameId'] == 400554231 ]
# Group by offense, calc what percentage of drives end in TD/FG/etc.
ratios = subdf.groupby(['offense'])[['TD','FG','punt','turnover','EoH']].mean()
np.round(ratios, decimals=3)

,TD,FG,punt,turnover,EoH
offense,,,,,
DEN,0.250,0.083,0.667,0.000,0.000
NYJ,0.154,0.077,0.538,0.077,0.154


In [15]:
# drop column Unnamed: 0 and save the dataFrame for future use.
all_drives.drop(['Unnamed: 0'],axis=1,inplace=True)
all_drives.sample(5)

,away,away_score_after,drive,home,home_score_after,offense,plays,result,time,yds_gained,gameId,uid,TD,FG,punt,turnover,EoH
14587,ARI,14,14,PIT,17,PIT,10,Touchdown,5:32,64,311023022,311023022-14,1,0,0,0,0
7188,MIA,20,14,NYJ,21,MIA,5,"Blocked Punt, Downs",1:06,19,300926015,300926015-14,0,0,1,1,0
46516,ATL,13,8,KC,13,ATL,1,Interception Touchdown,0:11,0,400874715,400874715-8,0,0,0,1,0
35220,NO,3,9,CAR,17,NO,8,Missed FG,3:31,31,400554388,400554388-9,0,1,0,0,0
15433,DAL,31,14,BUF,7,BUF,1,Intercepted Pass,0:00,19,311113006,311113006-14,0,0,0,1,0


In [16]:
all_drives.to_csv('../data/espn_drives2009-2016.csv')

In [18]:
subdf = all_drives[all_drives['offense']=='CAR']
ratios = subdf.groupby(['gameId'])[['TD','FG','punt','turnover','EoH']].mean()
np.round(ratios, decimals=3)

,TD,FG,punt,turnover,EoH
gameId,,,,,
290913029,0.067,0.067,0.267,0.533,0.067
290920001,0.200,0.200,0.200,0.200,0.200
290928006,0.091,0.000,0.545,0.273,0.091
291011029,0.182,0.091,0.273,0.273,0.182
291018027,0.364,0.000,0.273,0.273,0.000
291025029,0.125,0.062,0.500,0.250,0.062
291101022,0.231,0.154,0.462,0.000,0.154
291108018,0.182,0.182,0.273,0.364,0.000
291115029,0.364,0.000,0.364,0.000,0.182
